In [2]:
import numpy as np
import os, sys
from datetime import datetime
import requests
from requests.exceptions import HTTPError
%load_ext autoreload
%autoreload 2

In [3]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [4]:
def get_transformation_matrix(animal):
    try:
        url = f'https://activebrainatlas.ucsd.edu/activebrainatlas/rotation/{animal}/manual/2'
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        transformation_matrix = response.json()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

    return transformation_matrix


In [6]:
prep_id = 'DK52'

In [33]:
transformation = get_transformation_matrix(prep_id)
r = transformation['rotation']
t = transformation['translation']
r = np.asarray(r)
t = np.asarray(t)

In [8]:
# person_id = 2 = beth
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    # bili, do your transformation on x,y,section here
    row_dict[structure] = [row.x, row.y, row.section]


In [28]:
def brain_to_atlas_transform(brain_coord, r, t):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates. 
    """
    brain_coord = np.asarray(brain_coord)
    atlas_coord = brain_coord @ r + t 
    return atlas_coord


In [66]:
data.T

array([[[ 3.02792759e+04,  3.04190094e+04,  3.05242032e+04,
          2.03038925e+04,  2.00982645e+04,  2.14081240e+04,
          2.14153716e+04,  2.45689324e+04,  2.44015851e+04,
          2.62108741e+04,  2.62055708e+04,  2.99223642e+04,
          2.85545103e+04,  3.06715890e+04,  2.99977284e+04,
          2.69882722e+04,  2.68638874e+04,  2.06294392e+04,
          2.48674273e+04,  2.48090739e+04,  3.33122877e+04,
          3.28676851e+04,  2.07677297e+04,  2.08714889e+04,
          2.30094782e+04,  2.28587613e+04,  2.34428201e+04,
          1.75130179e+04,  1.97289231e+04,  1.97676448e+04,
          2.04695988e+04,  2.05202704e+04,  3.27587486e+04,
          3.20773286e+04,  2.97648118e+04,  2.96148730e+04,
          2.61105475e+04,  2.61983553e+04,  2.56453676e+04,
          2.51595708e+04,  2.71857161e+04,  2.75005237e+04,
          2.23361652e+04,  2.25852444e+04],
        [ 3.31633449e+04,  3.33030783e+04,  3.34082721e+04,
          2.31879614e+04,  2.29823334e+04,  2.42921930e+

In [86]:
r + t.T
RT = r + t.T

In [97]:
data = np.asarray([45542.0, 17755.0, 218.0])
data1 = data.reshape(1,3)
print(data1.shape)
print(t.shape)
print(r)
print(t)
data1 @ r + t

(1, 3)
(3, 1)
[[ 0.90823119 -0.24441544 -0.05697743]
 [ 0.2488724   0.90471918  0.08611017]
 [ 0.03237074 -0.09804858  0.93659374]]
[[-15509.17500957]
 [-12625.10609468]
 [   -66.471405  ]]


array([[ 30279.27594153, -10598.42878454, -16370.97775207],
       [ 33163.34485642,  -7714.35986964, -13486.90883717],
       [ 45721.9795461 ,   4844.27482003,   -928.2741475 ]])

In [76]:
data = []
person_id = 28 # bili, this is your database ID
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    print(x, y, section)
    x,y,section = brain_to_atlas_transform([x,y,section], r, t)
    data.append([x, y, section])
    continue
    com = CenterOfMass(prep_id=prep_id, structure=structure, x=x, y=y, section=section,
                   created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned')
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()

data = np.asarray(data)

45542.0 17755.0 218.0
45714.0 17685.0 247.0
45596.0 18540.0 234.0
34961.0 16285.0 232.0
34749.0 16231.0 243.0
36247.0 16030.0 223.0
36247.0 16056.0 247.0
38765.0 19548.0 172.0
38914.0 18314.0 309.0
40686.0 19124.0 257.0
40599.0 19425.0 220.0
44309.0 20809.0 307.0
42465.0 22042.0 309.0
46190.0 16965.0 230.0
44094.0 21914.0 172.0
41986.0 17520.0 130.0
42227.0 16113.0 343.0
37176.0 9511.0 222.0
39808.0 16939.0 188.0
39934.0 16233.0 278.0
47708.0 22042.0 183.0
47409.0 21332.0 296.0
35560.0 15975.0 138.0
35814.0 15440.0 330.0
35794.0 24118.0 220.0
35751.0 23661.0 284.0
36771.0 22290.0 249.0
33868.0 9061.0 222.0
33001.0 21134.0 182.0
33458.0 19604.0 319.0
34604.0 18256.0 214.0
34186.0 19971.0 322.0
48044.0 18595.0 157.0
47299.0 18553.0 332.0
44462.0 19638.0 150.0
44437.0 19104.0 325.0
39145.0 24349.0 223.0
39342.0 23976.0 276.0
39930.0 19629.0 116.0
39895.0 17773.0 360.0
41377.0 20537.0 121.0
42273.0 18501.0 360.0
36542.0 18691.0 157.0
37048.0 17823.0 328.0
